# CRIPTOATAQUES

In [2]:
from lib.ataques.cifra_de_vigenere.vigénere import VigenereCifra

## Cifra de César

## Análise de Frequência


## Cifra de Vigénere

In [3]:
vigenere = VigenereCifra()

In [ ]:
texto_original = open("", encoding="utf-8")
chave_encr = ""
texto_cifrado = vigenere.encriptar_decriptar(texto_original,chave_encr,"ENCRIPTAR")

In [ ]:
texto_cifrado

In [ ]:
tamanho_chave = vigenere.tamanho_chave(texto_cifrado)

In [ ]:
chave_encontrada = vigenere.quebra_chave(texto_cifrado,tamanho_chave, 'pt')

In [ ]:
texto_decriptado = vigenere.encriptar_decriptar(texto_cifrado, chave_encontrada,"DECRIPTAR")

## Criptografia RSA